# Train GPT-OSS 20B → Rust Coding Agent (v3 — Strandset)

Simplified pipeline using [Strandset-Rust-v1](https://huggingface.co/datasets/Fortytwo-Network/Strandset-Rust-v1) (191K verified Rust examples, Apache 2.0) as the sole data source.

**Key differences from v2:**
- **No Rust toolchain** — no `rustup`, `cargo-mutants`, or compilation needed
- **No mutation/trajectory generation** — data comes entirely from Strandset
- **No GRPO** — no execution-based rewards without cargo
- **IPO from synthetic preferences** — bug_detection pairs (fixed=chosen, buggy=rejected)

**3-Phase Pipeline:**
1. **Lang Adapter** — Rust domain specialisation via QLoRA (script 13 + 19)
2. **Core Agent SFT** — Debug/review training from Strandset (script 14)
3. **IPO Preference** — Synthetic preference pairs from bug_detection (script 17)

**Requirements:**
- **GPU**: A100 40GB+ (H100 80GB recommended for FP8)
- **Storage**: Google Drive for persistent checkpoints
- **No Rust toolchain required**

---
## Step 0: Environment Setup

### 0.1 Mount Google Drive & Clone Repository

**PyCharm / headless users:** If `drive.mount()` doesn't work, set `use_service_account = True`
and provide your service-account JSON key in Step 0.3.

In [ ]:
import os
import sys

try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

use_service_account = False  # Use drive.mount() in Colab UI

DRIVE_MOUNTED = False

if IN_COLAB and not use_service_account:
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        DRIVE_MOUNTED = True
        print("Google Drive mounted")
    except Exception as e:
        print(f"drive.mount() failed: {e}")
        print("Falling back to local-only mode.")
        print("Tip: set use_service_account=True and provide a JSON key in Step 0.3.")
elif IN_COLAB and use_service_account:
    print("Service-account mode selected \u2014 skipping drive.mount()")
    print("Configure credentials in Step 0.3.")
else:
    print("Running locally")

REPO_URL = "https://github.com/rmarnold/llm-training-pipeline.git"
BRANCH = "main"

REPO_DIR = "/content/llm-training-pipeline"

if IN_COLAB:
    if os.path.exists(REPO_DIR):
        %cd {REPO_DIR}
        !git pull origin {BRANCH}
    else:
        !git clone -b {BRANCH} {REPO_URL} {REPO_DIR}
        %cd {REPO_DIR}

    PROJECT_ROOT = REPO_DIR
else:
    PROJECT_ROOT = os.getcwd()

os.chdir(PROJECT_ROOT)
print(f"\nProject root: {PROJECT_ROOT}")

### 0.2 Install Dependencies

Installs pipeline deps and latest Unsloth. **No Rust toolchain needed** — all training data
comes from Strandset.

**Note:** flash-attn is intentionally NOT installed. FA3 is incompatible with GPT-OSS
backward passes. Unsloth's Flex Attention replaces it automatically.

In [2]:
if IN_COLAB:
    print("Installing Python dependencies...")
    print("=" * 60)
    !pip install -q -e ".[gpt_oss,colab]"

    # Fix pyarrow binary incompatibility with datasets 4.x on Colab
    !pip install -q --force-reinstall pyarrow

    # Force latest Unsloth with Split LoRA + FP8 RL
    print("\nInstalling latest Unsloth (Split LoRA + Flex Attention)...")
    !pip install -q --upgrade --force-reinstall --no-cache-dir --no-deps unsloth unsloth_zoo
    !pip install -q "unsloth[colab-new]"

    # vLLM for FP8 inference (H100 only, optional)
    !pip install -q vllm>=0.12.0 2>/dev/null || true

    # Verification
    from importlib.metadata import version, PackageNotFoundError
    print("\n" + "=" * 60)
    print("Dependency Verification:")
    print("=" * 60)

    for pkg in ["unsloth", "trl", "peft", "datasets", "tiktoken", "vllm"]:
        try:
            ver = version(pkg)
            print(f"\u2713 {pkg}: {ver}")
        except PackageNotFoundError:
            if pkg == "vllm":
                print(f"\u2014 {pkg}: not installed (optional, H100 FP8 only)")
            else:
                print(f"\u2717 {pkg}: not installed")

    print("\nNote: No Rust toolchain needed for v3 (Strandset-only pipeline)")
    print("=" * 60)
else:
    print("Running locally \u2014 ensure deps are installed:")
    print("  pip install -e '.[gpt_oss]'")
    print("  pip install --upgrade unsloth unsloth_zoo")

Installing Python dependencies...
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 98.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 141.5 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.3/432.3 kB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━

### 0.3 Configure Pipeline

**Training Scope** (`training_scope`):
- `full` \u2014 All 3 phases (Lang Adapter + Core Agent + IPO)
- `quick_test` \u2014 Short runs (100 steps each) to verify setup
- `lang_adapter_only` \u2014 Only train lang_rust adapter + merge

**Service Account Setup** (for Drive backup):
1. Set `use_service_account = True` in cell 0.1
2. Run cell 0.3 \u2014 it will try Colab Secrets, then file, then paste prompt
3. Set `DRIVE_FOLDER_ID` in Colab Secrets, or set `drive_folder_id` below

In [3]:
import json
training_scope = "quick_test"  # "full", "quick_test", "lang_adapter_only"

gpu_tier = "h100_80gb"  # "a100_40gb", "a100_80gb", "h100_80gb"

max_steps_override = 0  # Set >0 to cap all stages (0 = use defaults)

include_ipo = True  # False to skip IPO preference training

enable_qat_export = False  # True for MXFP4 QAT export

include_mutations = False  # True to run cargo-mutants in background (requires Rust toolchain)

# ============================================================
# SERVICE ACCOUNT CREDENTIALS
# ============================================================
# Priority order:
#   1. Existing VALID file at /content/service_account.json (instant, no timeout)
#   2. Colab Secrets (only if no valid file — may timeout outside browser UI)
#   3. Paste JSON key via input() prompt
#   4. Fall back to local mode (no Drive backup)

drive_folder_id = "18UpFpUhiNrs2Etha0uFjSGWmj1Ee1SnX"  # Google Drive folder ID

_SA_VM_PATH = "/content/service_account.json"
_FOLDER_ID_PATH = "/content/drive_folder_id.txt"
service_account_key = ""

def _is_json(s):
    """Check if string looks like JSON (not a folder ID)."""
    return s.strip().startswith("{")

def _validate_sa_file(path):
    """Check that a service account JSON file exists and contains valid JSON."""
    try:
        with open(path) as f:
            data = json.load(f)
        return isinstance(data, dict) and "type" in data
    except (json.JSONDecodeError, OSError, ValueError):
        return False

if use_service_account and IN_COLAB:
    # 1. Check for existing VALID file first (avoids Colab Secrets timeout on re-runs)
    if os.path.exists(_SA_VM_PATH) and _validate_sa_file(_SA_VM_PATH):
        service_account_key = _SA_VM_PATH
        print(f"Using existing key file: {_SA_VM_PATH}")
    else:
        if os.path.exists(_SA_VM_PATH):
            os.remove(_SA_VM_PATH)
            print(f"Removed invalid/empty key file: {_SA_VM_PATH}")

        # 2. Try Colab Secrets (may timeout if not running in browser UI)
        try:
            from google.colab import userdata
            _key_json = userdata.get("SERVICE_ACCOUNT_KEY")
            if _key_json:
                # Validate before saving
                json.loads(_key_json)
                with open(_SA_VM_PATH, "w") as _f:
                    _f.write(_key_json)
                service_account_key = _SA_VM_PATH
                print("Service account key loaded from Colab Secrets.")
        except json.JSONDecodeError:
            print("  Colab Secret SERVICE_ACCOUNT_KEY contains invalid JSON.")
        except Exception as _e:
            print(f"  Colab Secrets lookup failed: {type(_e).__name__}: {_e}")

        # 3. Fall back to paste prompt
        if not service_account_key:
            try:
                print("No service account key found.")
                _key_text = input("Paste service account JSON (entire content in one go): ")
                _key_text = _key_text.strip()
                if _key_text:
                    json.loads(_key_text)
                    with open(_SA_VM_PATH, "w") as _f:
                        _f.write(_key_text)
                    service_account_key = _SA_VM_PATH
                    print(f"Saved to {_SA_VM_PATH}")
            except json.JSONDecodeError:
                print("  Invalid JSON — key not saved.")
            except EOFError:
                pass

    # Resolve drive_folder_id: saved file > Colab Secrets > input prompt
    if not drive_folder_id and os.path.exists(_FOLDER_ID_PATH):
        with open(_FOLDER_ID_PATH) as _f:
            drive_folder_id = _f.read().strip()
        if drive_folder_id:
            print(f"Using saved folder ID from {_FOLDER_ID_PATH}")

    if not drive_folder_id:
        try:
            from google.colab import userdata
            _fid = userdata.get("DRIVE_FOLDER_ID") or ""
            if _fid and not _is_json(_fid):
                drive_folder_id = _fid
                print(f"Drive folder ID loaded from Colab Secrets.")
            elif _fid:
                print("WARNING: DRIVE_FOLDER_ID Colab Secret contains JSON, not a folder ID. Ignoring.")
        except Exception:
            pass

    if not drive_folder_id and service_account_key:
        _fid = input("Enter Google Drive folder ID (from URL): ").strip()
        if _fid and not _is_json(_fid):
            drive_folder_id = _fid
            # Persist so we don't have to re-enter on re-runs
            with open(_FOLDER_ID_PATH, "w") as _f:
                _f.write(drive_folder_id)
            print(f"Saved folder ID to {_FOLDER_ID_PATH}")
        elif _fid:
            print("ERROR: That looks like JSON, not a folder ID.")
            print("The folder ID is the part after /folders/ in the Google Drive URL.")

    if not service_account_key:
        print("No service account key — Drive backup disabled.")

elif use_service_account:
    for _path in [_SA_VM_PATH, "service_account.json"]:
        if os.path.exists(_path):
            service_account_key = _path
            print(f"Using key file: {_path}")
            break
    if not service_account_key:
        print("Running locally — set service_account_key to your JSON key path.")

# ============================================================
# DRIVE MODE
# ============================================================
import importlib
import scripts.pipeline_lib.drive_utils as _du_mod
importlib.reload(_du_mod)  # pick up any fixes from git pull
from scripts.pipeline_lib.drive_utils import DriveHelper

DRIVE_BASE = "/content/drive/MyDrive/gpt-oss-20b-rust-agent-v3"

if DRIVE_MOUNTED:
    DRIVE_MODE = "mounted"
elif use_service_account and service_account_key and drive_folder_id:
    DRIVE_MODE = "service_account"
else:
    DRIVE_MODE = "local"

drive_helper = DriveHelper(
    mode=DRIVE_MODE,
    drive_base=DRIVE_BASE,
    credentials_path=service_account_key or None,
    folder_id=drive_folder_id or None,
)

# ============================================================
# GPU TIER CONFIGS
# ============================================================

GPU_CONFIGS = {
    "a100_40gb": {
        "moe_backend": "unsloth_triton",
        "load_mode": "4bit",
        "fast_inference": False,
        "lang_rust": {"batch": 1, "grad_accum": 8, "seq_len": 8192, "max_steps": 3000},
        "core_agent": {"batch": 1, "grad_accum": 4, "seq_len": 12288, "max_steps": 2000},
        "ipo": {"batch": 1, "grad_accum": 8, "seq_len": 12288, "max_steps": 1000},
    },
    "a100_80gb": {
        "moe_backend": "unsloth_triton",
        "load_mode": "4bit",
        "fast_inference": False,
        "lang_rust": {"batch": 1, "grad_accum": 8, "seq_len": 8192, "max_steps": 5000},
        "core_agent": {"batch": 1, "grad_accum": 4, "seq_len": 16384, "max_steps": 3000},
        "ipo": {"batch": 1, "grad_accum": 16, "seq_len": 16384, "max_steps": 2000},
    },
    "h100_80gb": {
        "moe_backend": "grouped_mm",
        "load_mode": "fp8",
        "fast_inference": True,
        "lang_rust": {"batch": 4, "grad_accum": 2, "seq_len": 8192, "max_steps": 5000},
        "core_agent": {"batch": 2, "grad_accum": 2, "seq_len": 16384, "max_steps": 3000},
        "ipo": {"batch": 2, "grad_accum": 4, "seq_len": 16384, "max_steps": 2000},
    },
}

if training_scope == "quick_test":
    max_steps_override = 100

gpu_cfg = GPU_CONFIGS[gpu_tier]

# Mutation generation (optional, requires Rust toolchain)
if include_mutations:
    import multiprocessing
    import os as _os
    cpu_count = multiprocessing.cpu_count()
    try:
        _mem_bytes = _os.sysconf("SC_PAGE_SIZE") * _os.sysconf("SC_PHYS_PAGES")
        total_ram_gb = _mem_bytes / (1024**3)
        ram_based_jobs = max(1, int(total_ram_gb / 4))
    except (ValueError, OSError):
        total_ram_gb = 0
        ram_based_jobs = cpu_count
    cpu_based_jobs = max(1, cpu_count - 2)
    mutation_jobs = min(cpu_based_jobs, ram_based_jobs)
    mutation_repo_workers = max(1, mutation_jobs // 4)
else:
    mutation_jobs = 0
    mutation_repo_workers = 0

CONFIG = {
    "training_scope": training_scope,
    "gpu_tier": gpu_tier,
    "include_ipo": include_ipo,
    "enable_qat_export": enable_qat_export,
    "moe_backend": gpu_cfg["moe_backend"],
    "load_mode": gpu_cfg["load_mode"],
    "fast_inference": gpu_cfg["fast_inference"],
    # Lang adapter
    "lang_rust_batch": gpu_cfg["lang_rust"]["batch"],
    "lang_rust_grad_accum": gpu_cfg["lang_rust"]["grad_accum"],
    "lang_rust_seq_len": gpu_cfg["lang_rust"]["seq_len"],
    "lang_rust_max_steps": max_steps_override or gpu_cfg["lang_rust"]["max_steps"],
    # Core agent
    "core_agent_batch": gpu_cfg["core_agent"]["batch"],
    "core_agent_grad_accum": gpu_cfg["core_agent"]["grad_accum"],
    "core_agent_seq_len": gpu_cfg["core_agent"]["seq_len"],
    "core_agent_max_steps": max_steps_override or gpu_cfg["core_agent"]["max_steps"],
    # IPO
    "ipo_batch": gpu_cfg["ipo"]["batch"],
    "ipo_grad_accum": gpu_cfg["ipo"]["grad_accum"],
    "ipo_seq_len": gpu_cfg["ipo"]["seq_len"],
    "ipo_max_steps": max_steps_override or gpu_cfg["ipo"]["max_steps"],
    # Eval
    "eval_num_samples": 10 if training_scope == "quick_test" else 50,
    # Mutation generation (optional)
    "include_mutations": include_mutations,
    "max_mutations_per_repo": 50,
    "mutation_jobs": mutation_jobs,
    "mutation_repo_workers": mutation_repo_workers,
}

print("=" * 60)
print("PIPELINE CONFIGURATION (v3 \u2014 Strandset)")
print("=" * 60)
print(f"\nScope: {training_scope.upper()}")
print(f"GPU tier: {gpu_tier}")
print(f"MoE backend: {CONFIG['moe_backend']}")
print(f"Load mode: {CONFIG['load_mode']}")
print(f"Fast inference (vLLM): {CONFIG['fast_inference']}")
print(f"Include IPO: {include_ipo}")
print(f"Include mutations (background): {include_mutations}")
print(f"QAT export: {enable_qat_export}")
print(f"Drive mode: {DRIVE_MODE}")
if max_steps_override:
    print(f"Max steps override: {max_steps_override}")
print(f"\nLang Adapter:  batch={CONFIG['lang_rust_batch']} x grad_accum={CONFIG['lang_rust_grad_accum']}, seq={CONFIG['lang_rust_seq_len']}, steps={CONFIG['lang_rust_max_steps']}")
print(f"Core Agent:    batch={CONFIG['core_agent_batch']} x grad_accum={CONFIG['core_agent_grad_accum']}, seq={CONFIG['core_agent_seq_len']}, steps={CONFIG['core_agent_max_steps']}")
if include_ipo:
    print(f"IPO:           batch={CONFIG['ipo_batch']} x grad_accum={CONFIG['ipo_grad_accum']}, seq={CONFIG['ipo_seq_len']}, steps={CONFIG['ipo_max_steps']}")
print("=" * 60)

  Colab Secrets lookup failed: TimeoutException: Requesting secret SERVICE_ACCOUNT_KEY timed out. Secrets can only be fetched when running from the Colab UI.
No service account key found.
Saved to /content/service_account.json


PIPELINE CONFIGURATION (v3 — Strandset)

Scope: QUICK_TEST
GPU tier: h100_80gb
MoE backend: grouped_mm
Load mode: fp8
Fast inference (vLLM): True
Include IPO: True
QAT export: False
Drive mode: service_account
Max steps override: 100

Lang Adapter:  batch=2 x grad_accum=4, seq=8192, steps=100
Core Agent:    batch=1 x grad_accum=4, seq=16384, steps=100
IPO:           batch=1 x grad_accum=16, seq=16384, steps=100


### 0.4 Set Up Persistent Storage

In [4]:
DRIVE_SUBDIRS = [
    "checkpoints/lang_rust",
    "checkpoints/core_agent",
    "checkpoints/core_agent_ipo",
    "checkpoints/gpt-oss-20b-rust-merged",
    "data/rust/strandset",
    "logs",
]

if DRIVE_MODE == "mounted":
    print(f"Setting up storage at: {DRIVE_BASE}")
    for subdir in DRIVE_SUBDIRS:
        os.makedirs(os.path.join(DRIVE_BASE, subdir), exist_ok=True)

    for dir_name in ["checkpoints", "data", "logs"]:
        local_path = os.path.join(PROJECT_ROOT, dir_name)
        drive_path = os.path.join(DRIVE_BASE, dir_name)

        if os.path.exists(local_path) and not os.path.islink(local_path):
            !cp -r {local_path}/* {drive_path}/ 2>/dev/null || true
            !rm -rf {local_path}
        elif os.path.islink(local_path):
            os.unlink(local_path)

        os.symlink(drive_path, local_path)
        print(f"  {dir_name} -> Drive (mounted)")

elif DRIVE_MODE == "service_account":
    print("Setting up local storage + Drive API restore...")
    for subdir in DRIVE_SUBDIRS:
        os.makedirs(os.path.join(PROJECT_ROOT, subdir), exist_ok=True)
        drive_helper.ensure_dir(subdir)

    for dir_name in ["checkpoints", "data", "logs"]:
        local_path = os.path.join(PROJECT_ROOT, dir_name)
        if os.path.islink(local_path):
            os.unlink(local_path)
            os.makedirs(local_path, exist_ok=True)
        print(f"  {dir_name} -> local (backed up via Drive API)")

    print("\nRestoring existing data from Drive...")
    for subdir in DRIVE_SUBDIRS:
        local_target = os.path.join(PROJECT_ROOT, subdir)
        drive_helper.restore(subdir, local_target)
    print("Restore complete.")

else:
    for d in ["checkpoints", "data/rust", "logs"]:
        os.makedirs(d, exist_ok=True)
    print("Local directories created (no Drive backup).")

print("\nStorage ready!")

Setting up local storage + Drive API restore...
  checkpoints -> local (backed up via Drive API)
  data -> local (backed up via Drive API)
  logs -> local (backed up via Drive API)

Restoring existing data from Drive...
Restore complete.

Storage ready!


### 0.5 Check GPU & Configure MoE Backend

In [5]:
import torch

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    capability = torch.cuda.get_device_capability()
    is_h100 = "H100" in gpu_name or "H200" in gpu_name or "B200" in gpu_name

    CONFIG["use_fp8"] = capability[0] >= 9 and is_h100

    if is_h100:
        detected_tier = "h100_80gb"
    elif gpu_memory >= 70:
        detected_tier = "a100_80gb"
    else:
        detected_tier = "a100_40gb"

    if detected_tier != CONFIG["gpu_tier"]:
        print(f"NOTE: Auto-detected {detected_tier}, overriding configured {CONFIG['gpu_tier']}")
        CONFIG["gpu_tier"] = detected_tier
        gpu_cfg = GPU_CONFIGS[detected_tier]
        CONFIG["moe_backend"] = gpu_cfg["moe_backend"]
        CONFIG["load_mode"] = gpu_cfg["load_mode"]
        CONFIG["fast_inference"] = gpu_cfg["fast_inference"]

    os.environ["UNSLOTH_MOE_BACKEND"] = CONFIG["moe_backend"]

    print("=" * 60)
    print(f"GPU: {gpu_name} ({gpu_memory:.0f} GB)")
    print(f"Compute capability: {capability[0]}.{capability[1]}")
    print(f"Tier: {CONFIG['gpu_tier']}")
    print(f"\nSplit LoRA backend: {CONFIG['moe_backend']}")
    print(f"Load mode: {CONFIG['load_mode']}")
    print(f"FP8 available: {CONFIG['use_fp8']}")
    print(f"Fast inference (vLLM): {CONFIG['fast_inference']}")

    if gpu_memory < 40:
        print("\nWARNING: <40 GB VRAM. Long-context training (16K+) may OOM.")
    print("=" * 60)
else:
    print("No GPU detected!")
    CONFIG["use_fp8"] = False
    os.environ["UNSLOTH_MOE_BACKEND"] = "native_torch"

GPU: NVIDIA H100 80GB HBM3 (79 GB)
Compute capability: 9.0
Tier: h100_80gb

Split LoRA backend: grouped_mm
Load mode: fp8
FP8 available: True
Fast inference (vLLM): True


---
## Step 1: Data Preparation

Downloads Strandset-Rust-v1 from HuggingFace, parses the 15 task categories,
and formats everything in Harmony for each training stage.

### 1.1 Download & Format Strandset

In [6]:
max_samples = 500 if CONFIG["training_scope"] == "quick_test" else 0  # 0 = all

print("Downloading & formatting Strandset-Rust-v1...")
print("=" * 60)

cmd = "python scripts/20_prepare_strandset.py"
if max_samples:
    cmd += f" --max_samples {max_samples}"
if not CONFIG["include_ipo"]:
    cmd += " --no-preferences"

!{cmd}

drive_helper.backup("data/rust/strandset", "data/rust/strandset")
if DRIVE_MODE != "local":
    print("\nBacked up Strandset data to Drive.")


Preparing Strandset-Rust-v1
  Dataset: Fortytwo-Network/Strandset-Rust-v1
  Output: data/rust/strandset
  Max samples: 500

Loading dataset...
README.md: 6.93kB [00:00, 35.5MB/s]
data/train-00000-of-00001.parquet: 100% 109M/109M [00:01<00:00, 94.0MB/s] 
data/test-00000-of-00001.parquet: 100% 359k/359k [00:00<00:00, 1.14MB/s]
Generating train split: 100% 191008/191008 [00:00<00:00, 575187.72 examples/s]
Generating test split: 100% 225/225 [00:00<00:00, 66968.38 examples/s]
  Train split: 191,008 examples
  Test split: 225 examples

Processing 500 examples...

Saving datasets...
Saving the dataset (1/1 shards): 100% 419/419 [00:00<00:00, 276453.26 examples/s]
  lang_rust/train: 419 examples -> data/rust/strandset/lang_rust/train
Saving the dataset (1/1 shards): 100% 48/48 [00:00<00:00, 36242.41 examples/s]
  core_agent/train: 48 examples -> data/rust/strandset/core_agent/train
Saving the dataset (1/1 shards): 100% 3/3 [00:00<00:00, 1730.32 examples/s]
  ipo/train: 3 preference pairs -> 


Backed up Strandset data to Drive.


### 1.1b Generate Mutations (Optional, Background)

If `include_mutations=True`, starts `cargo-mutants` in the background during training.
Mutations are used to generate trajectories that enrich core_agent training data.
Requires Rust toolchain (`cargo`, `cargo-mutants`).

Monitor progress: `!tail -f logs/mutation_gen.log`

In [ ]:
import subprocess

mutation_proc = None  # Will hold the background process

if not CONFIG["include_mutations"]:
    print("Mutation generation disabled (include_mutations=False)")
    print("Set include_mutations=True in Step 0.3 to enrich core_agent data with trajectories.")
else:
    max_muts = CONFIG["max_mutations_per_repo"]
    jobs = CONFIG["mutation_jobs"]
    repo_workers = CONFIG["mutation_repo_workers"]

    # Determine backup dir for incremental per-repo backup to Drive
    backup_dir = None
    if DRIVE_MODE == "mounted":
        backup_dir = os.path.join(DRIVE_BASE, "data/rust/mutations")
        os.makedirs(backup_dir, exist_ok=True)

    print(f"Starting mutation generation in BACKGROUND...")
    print(f"  Max {max_muts}/repo, {jobs} jobs, {repo_workers} repo workers")
    if backup_dir:
        print(f"  Per-repo backup: {backup_dir}")
    print("=" * 60)

    os.makedirs("logs", exist_ok=True)

    mutation_cmd = [
        "python", "scripts/16_generate_mutations.py",
        "--max_mutations_per_repo", str(max_muts),
        "--jobs", str(jobs),
        "--repo-workers", str(repo_workers),
    ]
    if backup_dir:
        mutation_cmd += ["--backup-dir", backup_dir]

    mutation_proc = subprocess.Popen(
        mutation_cmd,
        stdout=open("logs/mutation_gen.log", "w"),
        stderr=subprocess.STDOUT,
    )
    print(f"Mutation generation started in background (PID: {mutation_proc.pid})")
    print(f"Monitor: !tail -f logs/mutation_gen.log")
    print(f"\nProceeding to training...")

### 1.2 Verify Data

In [7]:
data_checks = [
    ("Strandset lang_rust", "data/rust/strandset/lang_rust/train"),
    ("Strandset core_agent", "data/rust/strandset/core_agent/train"),
    ("Strandset IPO", "data/rust/strandset/ipo/train"),
    ("Strandset eval", "data/rust/strandset/eval/test"),
    ("Stats", "data/rust/strandset/stats.json"),
]

print("Data Verification:")
print("=" * 60)
for name, path in data_checks:
    exists = os.path.exists(path)
    if exists and os.path.isdir(path):
        items = os.listdir(path)
        print(f"  \u2713 {name}: {path} ({len(items)} items)")
    elif exists:
        size_kb = os.path.getsize(path) / 1024
        print(f"  \u2713 {name}: {path} ({size_kb:.1f} KB)")
    else:
        needed = True
        if not CONFIG["include_ipo"] and "IPO" in name:
            needed = False
        if CONFIG["training_scope"] == "lang_adapter_only" and name in ("Strandset core_agent", "Strandset IPO"):
            needed = False
        sym = "\u2717" if needed else "\u2014"
        label = "MISSING" if needed else "not needed"
        print(f"  {sym} {name}: {label}")

# Show stats if available
stats_path = "data/rust/strandset/stats.json"
if os.path.exists(stats_path):
    import json
    with open(stats_path) as f:
        stats = json.load(f)
    print(f"\n  Total processed: {stats.get('total_processed', '?'):,}")
    print(f"  Lang adapter: {stats.get('lang_rust', '?'):,}")
    print(f"  Core agent: {stats.get('core_agent_debug', 0) + stats.get('core_agent_review', 0):,}")
    print(f"  IPO pairs: {stats.get('ipo', '?'):,}")
print("=" * 60)

Data Verification:
  ✓ Strandset lang_rust: data/rust/strandset/lang_rust/train (3 items)
  ✓ Strandset core_agent: data/rust/strandset/core_agent/train (3 items)
  ✓ Strandset IPO: data/rust/strandset/ipo/train (3 items)
  ✓ Strandset eval: data/rust/strandset/eval/test (3 items)
  ✓ Stats: data/rust/strandset/stats.json (0.5 KB)

  Total processed: 500
  Lang adapter: 419
  Core agent: 48
  IPO pairs: 3


---
## Step 2: Lang Adapter Training

Train a QLoRA adapter (rank 64) to specialise GPT-OSS 20B on Rust syntax, stdlib, and idioms.
Uses Strandset's code_generation, code_completion, docstring, comment, and naming examples.
Then merge the adapter into the base weights for downstream training.

**Split LoRA** backend auto-enabled for 7-12x faster MoE training.

### 2.1 Train lang_rust Adapter

In [8]:
batch = CONFIG["lang_rust_batch"]
grad_accum = CONFIG["lang_rust_grad_accum"]
max_steps = CONFIG["lang_rust_max_steps"]
seq_len = CONFIG["lang_rust_seq_len"]

cmd = f"python scripts/13_train_lang_adapter.py"
cmd += f" --train_data_path data/rust/strandset/lang_rust/train"
cmd += f" --per_device_train_batch_size {batch}"
cmd += f" --gradient_accumulation_steps {grad_accum}"
cmd += f" --max_steps {max_steps}"

print(f"Training lang_rust adapter...")
print(f"  Data: data/rust/strandset/lang_rust/train")
print(f"  Batch: {batch} x {grad_accum} = {batch * grad_accum}")
print(f"  Max steps: {max_steps}")
print(f"  Seq length: {seq_len} (from config)")
print(f"  Split LoRA backend: {CONFIG['moe_backend']}")
print("=" * 60)

!{cmd}

drive_helper.backup("checkpoints/lang_rust", "checkpoints/lang_rust")
if DRIVE_MODE != "local":
    print("\nCheckpoint backed up to Drive.")

Training lang_rust adapter...
  Data: data/rust/strandset/lang_rust/train
  Batch: 2 x 4 = 8
  Max steps: 100
  Seq length: 8192 (from config)
  Split LoRA backend: grouped_mm

Training Language Adapter: gpt-oss-20b-lang-rust-v1

Loading model: openai/gpt-oss-20b
/content/llm-training-pipeline/scripts/pipeline_lib/unsloth_utils.py:38: UserWarning: WARNING: Unsloth should be imported before [transformers] to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
2026-02-14 07:59:13.293267: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE


Checkpoint backed up to Drive.


### 2.2 Merge lang_rust into Base

In [9]:
print("Merging lang_rust adapter into base model...")
print("=" * 60)

!python scripts/19_merge_adapter.py \
    --adapter_path checkpoints/lang_rust/final \
    --output_dir checkpoints/gpt-oss-20b-rust-merged \
    --export_formats hf

drive_helper.backup("checkpoints/gpt-oss-20b-rust-merged", "checkpoints/gpt-oss-20b-rust-merged")
if DRIVE_MODE != "local":
    print("\nMerged model backed up to Drive.")

Merging lang_rust adapter into base model...

Merging Adapter: merge-lang-rust
  Base model: openai/gpt-oss-20b
  Adapter: checkpoints/lang_rust/final
  Output: checkpoints/gpt-oss-20b-rust-merged
  Formats: ['hf']
/content/llm-training-pipeline/scripts/pipeline_lib/unsloth_utils.py:138: UserWarning: WARNING: Unsloth should be imported before [transformers] to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
2026-02-14 08:16:35.068071: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-14 08:16:35.081088: E e

### 2.3 Verify Merge

In [10]:
merged_path = "checkpoints/gpt-oss-20b-rust-merged"
adapter_path = "checkpoints/lang_rust/final"

print("Merge Verification:")
print("=" * 60)

if os.path.exists(merged_path):
    files = os.listdir(merged_path)
    safetensors = [f for f in files if f.endswith(".safetensors")]
    print(f"  \u2713 Merged model: {merged_path}")
    print(f"    {len(safetensors)} safetensors shard(s), {len(files)} total files")
else:
    print(f"  \u2717 Merged model not found at {merged_path}")

if os.path.exists(adapter_path):
    adapter_files = os.listdir(adapter_path)
    print(f"  \u2713 Adapter: {adapter_path} ({len(adapter_files)} files)")
else:
    print(f"  \u2717 Adapter not found at {adapter_path}")

if CONFIG["training_scope"] == "lang_adapter_only":
    print("\n\u2713 lang_adapter_only scope complete. Stopping here.")

print("=" * 60)

Merge Verification:
  ✓ Merged model: checkpoints/gpt-oss-20b-rust-merged
    0 safetensors shard(s), 0 total files
  ✓ Adapter: checkpoints/lang_rust/final (7 files)


### 2.4 Wait for Mutations + Generate Trajectories (Optional)

If `include_mutations=True`, waits for background mutation generation to finish,
then generates agent trajectories to enrich core_agent training data.

Skipped if mutations are disabled.

In [ ]:
if not CONFIG["include_mutations"]:
    print("Mutation generation disabled — skipping.")
elif CONFIG["training_scope"] == "lang_adapter_only":
    print(f"Skipping — scope is {CONFIG['training_scope']}")
else:
    # --- Wait for background mutation generation ---
    if mutation_proc is not None:
        if mutation_proc.poll() is None:
            print("Waiting for background mutation generation to finish...")
            print("(Monitor: !tail -f logs/mutation_gen.log)")
            mutation_proc.wait()

        rc = mutation_proc.returncode
        if rc == 0:
            print(f"Mutation generation completed successfully.")
        else:
            print(f"WARNING: Mutation generation failed (rc={rc})")
            print("Check logs/mutation_gen.log for details.")
            print("Continuing — trajectories will use whatever data is available.")
    else:
        print("No background mutation process found.")

    # Backup mutation results to Drive
    drive_helper.backup("data/rust/mutations", "data/rust/mutations")
    if DRIVE_MODE != "local":
        print("Backed up mutations to Drive.")

    # --- Generate trajectories to enrich core_agent data ---
    max_samples = 500 if CONFIG["training_scope"] == "quick_test" else 5000

    print(f"\nGenerating trajectories (max {max_samples} per source)...")
    print("=" * 60)

    cmd = f"python scripts/15_generate_trajectories.py --max_samples {max_samples}"

    mutations_path = "data/rust/mutations/mutations.jsonl"
    if os.path.exists(mutations_path):
        cmd += f" --mutations_path {mutations_path}"

    !{cmd}

    # Merge trajectory data into Strandset core_agent dataset
    from datasets import load_from_disk, concatenate_datasets

    traj_path = "data/rust/core_agent/train"
    strandset_core_path = "data/rust/strandset/core_agent/train"

    if os.path.exists(traj_path) and os.path.exists(strandset_core_path):
        traj_ds = load_from_disk(traj_path)
        strandset_ds = load_from_disk(strandset_core_path)
        print(f"\n  Enriching core_agent: {len(strandset_ds):,} Strandset + {len(traj_ds):,} trajectories")
        merged = concatenate_datasets([strandset_ds, traj_ds])
        merged.save_to_disk(traj_path)
        print(f"  Saved enriched dataset: {len(merged):,} total -> {traj_path}")

    drive_helper.backup("data/rust/core_agent", "data/rust/core_agent")
    if DRIVE_MODE != "local":
        print("\nBacked up trajectories to Drive.")

---
## Step 3: Core Agent SFT

Train a higher-rank LoRA adapter (rank 128) on Strandset's debug/review examples.
Uses the merged lang_rust model as the base.

**Split LoRA** + **Auto packing** (3x faster, zero-config).

### 3.1 Train core_agent Adapter

In [11]:
if CONFIG["training_scope"] == "lang_adapter_only":
    print("Skipping \u2014 scope is lang_adapter_only")
else:
    batch = CONFIG["core_agent_batch"]
    grad_accum = CONFIG["core_agent_grad_accum"]
    max_steps = CONFIG["core_agent_max_steps"]
    seq_len = CONFIG["core_agent_seq_len"]

    cmd = f"python scripts/14_train_core_agent.py"
    cmd += f" --train_data_path data/rust/strandset/core_agent/train"
    cmd += f" --per_device_train_batch_size {batch}"
    cmd += f" --gradient_accumulation_steps {grad_accum}"
    cmd += f" --max_steps {max_steps}"

    print(f"Training core_agent adapter...")
    print(f"  Data: data/rust/strandset/core_agent/train")
    print(f"  Batch: {batch} x {grad_accum} = {batch * grad_accum}")
    print(f"  Max steps: {max_steps}")
    print(f"  Seq length: {seq_len} (from config)")
    print(f"  LoRA rank: 128")
    print(f"  Split LoRA backend: {CONFIG['moe_backend']}")
    print(f"  Auto packing: enabled (uncontaminated)")
    print("=" * 60)

    !{cmd}

    drive_helper.backup("checkpoints/core_agent", "checkpoints/core_agent")
    if DRIVE_MODE != "local":
        print("\nCheckpoint backed up to Drive.")

Training core_agent adapter...
  Data: data/rust/strandset/core_agent/train
  Batch: 1 x 4 = 4
  Max steps: 100
  Seq length: 16384 (from config)
  LoRA rank: 128
  Split LoRA backend: grouped_mm
  Auto packing: enabled (uncontaminated)

Training Core Agent: gpt-oss-20b-core-agent-v1

Merging lang_rust adapter from checkpoints/lang_rust...
  (Run scripts/19_merge_adapter.py first, or use --no-merge-lang-adapter)
  Falling back to base model: openai/gpt-oss-20b

Loading model: openai/gpt-oss-20b
/content/llm-training-pipeline/scripts/pipeline_lib/unsloth_utils.py:38: UserWarning: WARNING: Unsloth should be imported before [transformers] to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
2026-02-14 08:17:10.964966: I tensorflow/


Checkpoint backed up to Drive.


### 3.2 Verify core_agent

In [12]:
if CONFIG["training_scope"] == "lang_adapter_only":
    print("Skipping \u2014 scope is lang_adapter_only")
else:
    ckpt_path = "checkpoints/core_agent/final"

    print("Core Agent Verification:")
    print("=" * 60)

    if os.path.exists(ckpt_path):
        files = os.listdir(ckpt_path)
        print(f"  \u2713 Checkpoint: {ckpt_path} ({len(files)} files)")

        adapter_config = os.path.join(ckpt_path, "adapter_config.json")
        if os.path.exists(adapter_config):
            import json
            with open(adapter_config) as f:
                cfg = json.load(f)
            print(f"    LoRA rank: {cfg.get('r', '?')}")
            print(f"    Alpha: {cfg.get('lora_alpha', '?')}")
            print(f"    Target modules: {cfg.get('target_modules', '?')}")
    else:
        print(f"  \u2717 Checkpoint not found at {ckpt_path}")

    print("=" * 60)

Core Agent Verification:
  ✓ Checkpoint: checkpoints/core_agent/final (7 files)
    LoRA rank: 128
    Alpha: 256
    Target modules: ['v_proj', 'q_proj', 'up_proj', 'gate_proj', 'o_proj', 'k_proj', 'down_proj']


---
## Step 4: IPO Preference Training (Optional)

Train with Identity Preference Optimisation on synthetic preference pairs
from Strandset's bug_detection category (fixed=chosen, buggy=rejected).

Very low learning rate (5e-7), 1 epoch only to avoid collapse.

Set `include_ipo=False` in Step 0.3 to skip.

### 4.1 Train with IPO

In [13]:
if CONFIG["training_scope"] == "lang_adapter_only":
    print("Skipping \u2014 scope is lang_adapter_only")
elif not CONFIG["include_ipo"]:
    print("Skipping \u2014 IPO disabled (include_ipo=False)")
else:
    batch = CONFIG["ipo_batch"]
    grad_accum = CONFIG["ipo_grad_accum"]
    max_steps = CONFIG["ipo_max_steps"]

    ipo_checkpoint = "checkpoints/core_agent/final"

    cmd = f"python scripts/17_ipo_preference.py"
    cmd += f" --checkpoint {ipo_checkpoint}"
    cmd += f" --train_data_path data/rust/strandset/ipo/train"
    cmd += f" --per_device_train_batch_size {batch}"
    cmd += f" --gradient_accumulation_steps {grad_accum}"
    cmd += f" --max_steps {max_steps}"

    print(f"Training with IPO (synthetic preferences)...")
    print(f"  Checkpoint: {ipo_checkpoint}")
    print(f"  Data: data/rust/strandset/ipo/train")
    print(f"  Batch: {batch} x {grad_accum} = {batch * grad_accum}")
    print(f"  Max steps: {max_steps}")
    print(f"  Loss: IPO (beta=0.1)")
    print(f"  Load mode: {CONFIG['load_mode']}")
    print(f"  Split LoRA backend: {CONFIG['moe_backend']}")
    print("=" * 60)

    !{cmd}

    drive_helper.backup("checkpoints/core_agent_ipo", "checkpoints/core_agent_ipo")
    if DRIVE_MODE != "local":
        print("\nCheckpoint backed up to Drive.")

Training with IPO (synthetic preferences)...
  Checkpoint: checkpoints/core_agent/final
  Data: data/rust/strandset/ipo/train
  Batch: 1 x 16 = 16
  Max steps: 100
  Loss: IPO (beta=0.1)
  Load mode: fp8
  Split LoRA backend: grouped_mm

IPO Preference Training: gpt-oss-20b-ipo-v1

Loading model from: checkpoints/core_agent/final
/content/llm-training-pipeline/scripts/pipeline_lib/unsloth_utils.py:38: UserWarning: WARNING: Unsloth should be imported before [transformers] to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
2026-02-14 08:32:26.890015: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computati


Checkpoint backed up to Drive.


### 4.2 Verify IPO

In [14]:
if CONFIG["training_scope"] == "lang_adapter_only":
    print("Skipping \u2014 scope is lang_adapter_only")
elif not CONFIG["include_ipo"]:
    print("Skipping \u2014 IPO disabled")
else:
    ckpt_path = "checkpoints/core_agent_ipo/final"

    print("IPO Verification:")
    print("=" * 60)

    if os.path.exists(ckpt_path):
        files = os.listdir(ckpt_path)
        print(f"  \u2713 IPO checkpoint: {ckpt_path} ({len(files)} files)")
    else:
        print(f"  \u2717 IPO checkpoint not found at {ckpt_path}")

    tb_dir = "checkpoints/core_agent_ipo"
    tb_files = []
    if os.path.exists(tb_dir):
        for root, dirs, fnames in os.walk(tb_dir):
            for fn in fnames:
                if fn.startswith("events.out.tfevents"):
                    tb_files.append(os.path.join(root, fn))
    if tb_files:
        print(f"  \u2713 TensorBoard logs found ({len(tb_files)} event files)")
        print(f"    Monitor KL divergence: warn >0.3, abort >0.5")
    else:
        print(f"  \u2014 No TensorBoard logs found")

    print("=" * 60)

IPO Verification:
  ✓ IPO checkpoint: checkpoints/core_agent_ipo/final (7 files)
  ✓ TensorBoard logs found (1 event files)
    Monitor KL divergence: warn >0.3, abort >0.5


---
## Step 5: Test Model

Load the trained model and generate Rust code interactively.

### 5.1 Load Model

In [2]:
from unsloth import FastLanguageModel
from peft import PeftModel
import torch

CHECKPOINT_PRIORITY = [
    "checkpoints/core_agent_ipo/final",
    "checkpoints/core_agent/final",
    "checkpoints/lang_rust/final",
]

# Merged model can be loaded directly (no adapter needed)
MERGED_PATH = "checkpoints/gpt-oss-20b-rust-merged"

MODEL_PATH = None
is_adapter = False
for path in CHECKPOINT_PRIORITY:
    if os.path.exists(path) and os.path.exists(os.path.join(path, "adapter_config.json")):
        MODEL_PATH = path
        is_adapter = True
        break

if MODEL_PATH is None and os.path.exists(MERGED_PATH):
    MODEL_PATH = MERGED_PATH
    is_adapter = False

if MODEL_PATH is None:
    print("\u2717 No checkpoint found. Train the model first.")
else:
    print(f"Loading model from: {MODEL_PATH}")
    print(f"  Type: {'LoRA adapter' if is_adapter else 'merged model'}")

    # GPT-OSS fused MoE experts (GptOssExperts) don't support BNB 4-bit
    # quantization at load time (no `down_projs` sub-module for traversal).
    # See: https://github.com/unslothai/unsloth/issues/3775
    #
    # Strategy: use Unsloth's pre-quantized BNB model (already 4-bit, no
    # quantizer runs at load). Falls back to bfloat16 if unavailable.
    model = None
    base_name = "openai/gpt-oss-20b"

    # Try 1: Pre-quantized BNB 4-bit (recommended by Unsloth BNB inference notebook)
    try:
        print("  Loading pre-quantized BNB 4-bit model...")
        model, tokenizer = FastLanguageModel.from_pretrained(
            "unsloth/gpt-oss-20b-unsloth-bnb-4bit",
            max_seq_length=4096,
            dtype=None,
            load_in_4bit=False,  # Already quantized — don't re-quantize
        )
        print("  Mode: BNB 4-bit (pre-quantized)")
    except Exception as e:
        print(f"  Pre-quantized BNB failed: {e}")

    # Try 2: bfloat16 without quantization (~40GB, fits H100 80GB)
    if model is None:
        print("  Loading in bfloat16 (no quantization)...")
        model, tokenizer = FastLanguageModel.from_pretrained(
            base_name,
            max_seq_length=4096,
            dtype=torch.bfloat16,
            load_in_4bit=False,
        )
        print("  Mode: bfloat16 (no quantization)")

    print("=" * 60)

    if is_adapter:
        print(f"  Applying LoRA adapter from {MODEL_PATH}...")
        model = PeftModel.from_pretrained(model, MODEL_PATH)

    FastLanguageModel.for_inference(model)
    print("\u2713 Model loaded!")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Loading model from: checkpoints/core_agent_ipo/final
  Type: LoRA adapter
  Loading pre-quantized BNB 4-bit model...
==((====))==  Unsloth 2026.2.1: Fast Gpt_Oss patching. Transformers: 4.57.6. vLLM: 0.15.1.
   \\   /|    NVIDIA H100 80GB HBM3. Num GPUs = 1. Max memory: 79.179 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 9.0. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00000-of-00002.safetensors:   0%|          | 0.00/4.79G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.80G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.17G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

  Mode: BNB 4-bit (pre-quantized)
  Applying LoRA adapter from checkpoints/core_agent_ipo/final...
Unsloth: PEFT set target_parameters but found no matching parameters.
This is expected for MoE models - Unsloth handles MoE expert LoRA targeting separately.
✓ Model loaded!


### 5.2 Generate Rust Code

In [ ]:
import sys
sys.path.insert(0, "scripts")
from dataset_formatters.harmony import encode_harmony_messages

TEST_PROMPTS = [
    "Write a Rust function `fn merge_sorted(a: &[i32], b: &[i32]) -> Vec<i32>` that merges two sorted slices into a single sorted vector.",
    "This Rust code fails the borrow checker. Fix it:\n```rust\nfn main() {\n    let mut v = vec![1, 2, 3];\n    let first = &v[0];\n    v.push(4);\n    println!(\"{}\", first);\n}\n```",
    "Write an async Rust function using tokio that fetches a URL with reqwest, retries up to 3 times on failure, and returns the response body as a String.",
]

def generate_rust(prompt, max_tokens=1024):
    messages = [{"role": "user", "content": prompt}]
    formatted = encode_harmony_messages(
        messages,
        developer_instructions="You are a Rust programming expert. Write correct, idiomatic code.",
        add_generation_prompt=True,
    )
    inputs = tokenizer(formatted, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=0.3,
            do_sample=True,
            top_p=0.9,
        )
    return tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)

for i, prompt in enumerate(TEST_PROMPTS, 1):
    print(f"\n{'=' * 60}")
    print(f"Test {i}: {prompt[:80]}...")
    print("=" * 60)
    response = generate_rust(prompt)
    print(response)
    print()

### 5.3 Custom Prompt

In [4]:
CUSTOM_PROMPT = "Write a Rust function that reads a CSV file and returns the sum of a specified column."

print(f"Prompt: {CUSTOM_PROMPT}")
print("=" * 60)
print(generate_rust(CUSTOM_PROMPT))

Prompt: Write a Rust function that reads a CSV file and returns the sum of a specified column.
﻿using System;
using System.Collections.Generic;
using System.Linq;
using System.Text;
using System.Threading.Tasks;

namespace _04._Theatre_Promotion
{
    internal class Program
    {
        static void Main(string[] args)
        {
            string day = Console.ReadLine();
            int age = int.Parse(Console.ReadLine());

            double price = 0;

            if (age < 0 || age > 122)
            {
                Console.WriteLine("Error!");
                return;
            }

            switch (day)
            {
                case "Weekday":
                    if (age <= 18 || age >= 64)
                        price = 12;
                    else
                        price = 18;
                    break;
                case "Weekend":
                    if (age <= 18 || age >= 64)
                        price = 15;
                    else
                   

---
## Step 6: Export

Merge the final adapter and export to HuggingFace + GGUF formats.

### 6.1 Export to GGUF

In [2]:
ADAPTER_PRIORITY = [
    "checkpoints/core_agent_ipo/final",
    "checkpoints/core_agent/final",
    "checkpoints/lang_rust/final",
]

adapter_path = None
for path in ADAPTER_PRIORITY:
    if os.path.exists(path):
        adapter_path = path
        break

if adapter_path is None:
    print("\u2717 No adapter checkpoint found.")
else:
    export_dir = "checkpoints/gpt-oss-20b-rust-export-v3"
    print(f"Exporting adapter: {adapter_path}")
    print(f"Output: {export_dir}")
    print("=" * 60)

    !python scripts/19_merge_adapter.py \
        --adapter_path {adapter_path} \
        --output_dir {export_dir} \
        --export_formats hf gguf_q4

    drive_helper.backup(export_dir, "checkpoints/gpt-oss-20b-rust-export-v3")
    if DRIVE_MODE != "local":
        print("\nExport backed up to Drive.")

Exporting adapter: checkpoints/core_agent_ipo/final
Output: checkpoints/gpt-oss-20b-rust-export-v3

Merging Adapter: merge-lang-rust
  Base model: openai/gpt-oss-20b
  Adapter: checkpoints/core_agent_ipo/final
  Output: checkpoints/gpt-oss-20b-rust-export-v3
  Formats: ['hf', 'gguf_q4']
/content/llm-training-pipeline/scripts/pipeline_lib/unsloth_utils.py:138: UserWarning: WARNING: Unsloth should be imported before [transformers] to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
2026-02-14 09:11:13.195345: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the enviro

NameError: name 'drive_helper' is not defined

### 6.2 Download GGUF

In [3]:
if IN_COLAB:
    from google.colab import files
    import glob

    export_dir = "checkpoints/gpt-oss-20b-rust-export-v3"
    gguf_files = glob.glob(os.path.join(export_dir, "*.gguf"))

    if gguf_files:
        gguf_path = gguf_files[0]
        size_gb = os.path.getsize(gguf_path) / (1024**3)
        print(f"Downloading: {os.path.basename(gguf_path)} ({size_gb:.1f} GB)")
        files.download(gguf_path)
    else:
        print("\u2717 No GGUF file found. Run export (6.1) first.")
else:
    print("Download not available outside Colab.")
    print("GGUF file is at: checkpoints/gpt-oss-20b-rust-export-v3/")

✗ No GGUF file found. Run export (6.1) first.


### 6.3 Upload to HuggingFace Hub

In [ ]:
# --- Configuration ---
HF_REPO_ID = ""  # e.g. "your-username/gpt-oss-20b-rust-agent-v3"
HF_PRIVATE = True

assert HF_REPO_ID, "Set HF_REPO_ID above before running this cell."

import glob
from huggingface_hub import HfApi

# Authenticate: try Colab Secrets first, then interactive login
try:
    from google.colab import userdata
    hf_token = userdata.get("HF_TOKEN")
    print("Using HF_TOKEN from Colab Secrets.")
except Exception:
    from huggingface_hub import login
    login()
    hf_token = None  # login() stores token globally

api = HfApi(token=hf_token)

# Create repo (no-op if it already exists)
api.create_repo(repo_id=HF_REPO_ID, private=HF_PRIVATE, exist_ok=True)
print(f"Repo ready: https://huggingface.co/{HF_REPO_ID}")

# --- Model card ---
export_dir = "checkpoints/gpt-oss-20b-rust-export-v3"
hf_dir = os.path.join(export_dir, "hf")

model_card = """\
---
base_model: openai/gpt-oss-20b
tags:
  - rust
  - code-agent
  - gpt-oss
  - qlora
  - unsloth
license: apache-2.0
pipeline_tag: text-generation
---

# GPT-OSS 20B Rust Agent (v3 — Strandset)

Fine-tuned from [openai/gpt-oss-20b](https://huggingface.co/openai/gpt-oss-20b) using the
[Strandset-Rust-v1](https://huggingface.co/datasets/Fortytwo-Network/Strandset-Rust-v1) dataset (191K examples).

## Training Pipeline

1. **Lang Adapter** — Rust domain specialisation (code generation/completion)
2. **Core Agent SFT** — Debug and review training (bug detection/code review)
3. **IPO** — Synthetic preference pairs (if enabled)

Trained with [Unsloth](https://github.com/unslothai/unsloth) QLoRA on NVIDIA H100 80GB.

## Usage

```python
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("{repo_id}")
tokenizer = AutoTokenizer.from_pretrained("{repo_id}")
```

## GGUF

A quantised GGUF file is included for use with llama.cpp or Ollama.
""".format(repo_id=HF_REPO_ID)

readme_path = os.path.join(hf_dir, "README.md")
with open(readme_path, "w") as f:
    f.write(model_card)
print(f"Wrote model card to {readme_path}")

# --- Upload HF safetensors model ---
assert os.path.isdir(hf_dir), f"HF export dir not found: {hf_dir}. Run export (6.1) first."
print(f"Uploading HF model from {hf_dir} ...")
api.upload_folder(
    folder_path=hf_dir,
    repo_id=HF_REPO_ID,
    commit_message="Upload merged HF model (v3 — Strandset pipeline)",
    token=hf_token,
)
print("HF model uploaded.")

# --- Upload GGUF file ---
gguf_files = glob.glob(os.path.join(export_dir, "**/*.gguf"), recursive=True)
if gguf_files:
    gguf_path = gguf_files[0]
    gguf_name = os.path.basename(gguf_path)
    size_gb = os.path.getsize(gguf_path) / (1024**3)
    print(f"Uploading GGUF: {gguf_name} ({size_gb:.1f} GB) ...")
    api.upload_file(
        path_or_fileobj=gguf_path,
        path_in_repo=gguf_name,
        repo_id=HF_REPO_ID,
        commit_message=f"Upload GGUF quantisation ({gguf_name})",
        token=hf_token,
    )
    print("GGUF uploaded.")
else:
    print("No GGUF file found — skipping. Run export (6.1) to generate one.")

print(f"\nDone! View your model at: https://huggingface.co/{HF_REPO_ID}")

---
## Training Complete!

Your GPT-OSS 20B Rust coding agent (v3 \u2014 Strandset) is trained and ready to use.

**Data source:** [Strandset-Rust-v1](https://huggingface.co/datasets/Fortytwo-Network/Strandset-Rust-v1) (191K examples, Apache 2.0)

**Pipeline:**
1. Lang Adapter: Rust domain specialisation from code generation/completion examples
2. Core Agent SFT: Debug and review training from bug_detection/code_review examples
3. IPO: Synthetic preference pairs from bug_detection (if enabled)

**Outputs:**
- Checkpoints: `checkpoints/core_agent_{ipo}/final`
- Exported model: `checkpoints/gpt-oss-20b-rust-export-v3/`
- All backed up to Google Drive: `gpt-oss-20b-rust-agent-v3/`

**Compared to v2:**
- No Rust toolchain required \u2014 runs on any Colab GPU instance
- No cargo-mutants or trajectory generation \u2014 faster setup
- No GRPO RL \u2014 no execution-based rewards
- For better results, consider upgrading to v2 with mutation data + GRPO